# Crypto Homework 4

In [ ]:
import numpy as np
import hashlib as hlib

In [ ]:
from provided_params import *
from provided_sol import *

### Ex 1: Stop the counter! (ECDSA)

We use the `secp256k1` standard for the parameters of the curve. The curve is given by $y^2=x^3+ 7$

Provided parameters:
* `Q1_G`: point used as a generator for the EC?
* `Q1_p`: a prime int used to define the finite field we are working in `GF(p)`
* `Q1_pk`: pair of int used as the secretary's public key
* 
* `Q1a_arr`: 30 triplets containing: ("X voted for Y" as a msg, r, s)
* `Q1a_msg`: SOLUTION
* 
* `Q1b_arr`: 2 valid ballots that were published after the counter was stopped
* `Q1b_k`: An int to select a unique solution
* `Q1b_r`: SOLUTION
* `Q1b_s`: SOLUTION

#TODO: Delete the next lines for submission

* [Inspiration for SAGE](https://stackoverflow.com/questions/40434409/rdsa-implementation-on-sage)
* Ex 1b is the attack against Sony. The r's of 2 messages are the same ==> reuse of a nonce ==> retrieve the secret key: [implemented attack](https://github.com/Marsh61/ECDSA-Nonce-Reuse-Exploit-Example/blob/master/Attack-Main.py)
* Something else?

In [ ]:
# Provided code example
def msg2int(m):
    h_bytes = hlib.sha256(m.encode()).digest()
    h = int.from_bytes(h_bytes, "big")
    return h

In [ ]:
# Initialization
EC = EllipticCurve(GF(Q1_p), [0,7])
FF_n = FiniteField(EC.order())

#### Question a

Identify the frauders

In [ ]:
def verify_sign(EC, msg, r, s, G, Q, FF_n):
    # Build point G and Q
    G_EC = EC(G)
    Q_EC = EC(Q)
    
    # Hash the message and covert it to number
    h = msg2int(msg)

    # Compute w = 1/s mod p because we use it twice
    w = Integer(1/FF_n(s))
    
    # Compute verification
    verif = w*h*G_EC + w*r*Q_EC
    
    verif_x = verif.xy()[0]
    
    return r == verif_x

In [ ]:
# Check the signature of every messages
for msg, r, s in Q1a_arr:
    verif_ok = verify_sign(EC, msg, r, s, Q1_G, Q1_pk, FF_n)
    if not verif_ok:
        print(msg)

#### Question b

The main problem here is: How to find `d` from the secret value and the counter? I fact, when we have `d` the problem is utterly easy.

```
h0 = Hash(message_0)
h1 = Hash(message_1)
d = Private Key (unknown to attacker)
R  = r1 == r2
K  = K value that was used (unknown to attacker)
N  = integer order of G (part of public key)

From Signing Defintion
s0 = (h0 + d * R) / K Mod N
s1 = (h1 + d * R) / K Mod N

Rearrange 
K = (h0 + d * R) / s0 Mod N
K = (h1 + d * R) / s1 Mod N

Set Equal
(h0 + d * R) / s0 = (h1 + d * R) / s1     Mod N

s1*h0 + s1*d*R = s0*h1 + s0*d*R     Mod N

(s1*h0 - s0*h1) / (R * (s0-s1)) = d     Mod N

Solve for d (private key)
d Mod N = (s1 * h0 - s0 * h1) / (R * (s0 - s1))
d Mod N = (s1 * h0 - s0 * h1) * (R * (s0 - s1)) ** -1
```

In [ ]:
def compute_r(EC, k, G, FF_n):
    return FF_n((k * EC(G)).xy()[0])

In [ ]:
def extract_d(ballots, FF_n):
    assert(len(ballots) == 2)
    msg0, r0, s0 = ballots[0]
    msg1, r1, s1 = ballots[1]
    
    h0 = msg2int(msg0)
    h1 = msg2int(msg1)
    
    s0 = FF_n(s0)
    s1 = FF_n(s1)
    
    # Verify if the attack is feasible
    if r0 != r1:
        raise Exception('ERROR: the messages you provided are not sensible to this attack')
    
    d = (s1*h0 - s0*h1) / (r0 *(s0 - s1))
    
    return d

In [ ]:
def compute_s(h, d, r, k, FF_n):
    return FF_n((h + d*r)/k)

In [ ]:
r = compute_r(EC, Q1b_k, Q1_G, FF_n)

# Hash the message and build an integer from that
m = 'I won this election, by a lot!'
h = msg2int(m)

# Find d
d = extract_d(Q1b_arr, FF_n)

# Compute s
s = compute_s(h, d, r, Q1b_k, FF_n)

In [ ]:
#TODO: Delete this verification for the submission
print(r)
print(Q1b_r)
print(Q1b_r == r)

print()
print(s)
print(Q1b_s)
print(Q1b_s == s)

##### SOME TESTS
Let's try to compare the `d`'s of the two provided valid messages.

---

### Ex 2

---
### Ex 3